In [75]:
import pandas as pd
import re
import gseapy as gp
import rpy2.robjects as ro
from tqdm.notebook import tqdm
from rpy2.robjects.conversion import localconverter
from rpy2.robjects import pandas2ri

In [55]:
gene_symbols = pd.read_csv('genes1.tsv', sep='\t')['rownames(eset)'].tolist()

In [57]:
id_map_r = ro.r("""
library(org.Hs.eg.db)
function (symbols) { 
    return(select(org.Hs.eg.db, symbols, c("ENTREZID"), "ALIAS"))
}
""")
with localconverter(ro.default_converter + pandas2ri.converter):
    id_map = id_map_r(ro.vectors.StrVector(gene_symbols))
id_map_clear = id_map.drop_duplicates('ENTREZID', keep='first').set_index('ALIAS')['ENTREZID'].to_dict()

R[write to console]: 'select()' returned 1:many mapping between keys and columns



In [63]:
dataset_list = list()
for i in range(13):
    expressions = pd.read_csv(f'expr{i + 1}.tsv', sep='\t')
    expressions['gene'] = pd.read_csv(f'genes{i + 1}.tsv', sep='\t')['rownames(eset)']
    expressions['gene'] = expressions['gene'].apply(lambda x: id_map_clear.get(x))
    expressions = expressions.loc[~expressions['gene'].isna()]
    expressions.set_index('gene', inplace=True, drop=True)
    expressions = expressions.T
    expressions.index.rename('sample', inplace=True)
    expressions['sample_type'] = pd.read_csv(f'anno{i + 1}.tsv', sep='\t')['Group'].tolist()
    dataset_list.append(expressions)

In [65]:
gensets_r = ro.r("""
library('EnrichmentBrowser')
getGenesets('hsa', 'kegg')
""")
gensets = {item[0]: list(item[1]) for item in zip(gensets_r.names, gensets_r)}

R[write to console]: Using cached version from 2020-12-21 11:53:08



In [96]:
target_keys = {key for key in gensets.keys() if re.search('[cC]ancer|[cC]arcinoma', key) is not None}
target_keys = target_keys.difference({
    'hsa05200_Pathways_in_cancer',
    'hsa05202_Transcriptional_misregulation_in_cancer',
    'hsa05205_Proteoglycans_in_cancer',
    'hsa05206_MicroRNAs_in_cancer',
    'hsa05235_PD-L1_expression_and_PD-1_checkpoint_pathway_in_cancer',
    'hsa05231_Choline_metabolism_in_cancer',
    'hsa05230_Central_carbon_metabolism_in_cancer'
})

In [97]:
target_gensets = {
    key: value for key, value in gensets.items() if key in target_keys
}

In [99]:
raw_gsea_result = list()
for i, dataset in tqdm(enumerate(dataset_list)):
    raw_gsea_result.append(
        gp.gsea(
            dataset.iloc[:, :-1].T,
            gene_sets=target_gensets,
            cls=dataset['sample_type'].values,
            permutation_num=1000,
            outdir=f"gsea-{i + 1}",
            permutation_type="phenotype",
            method="signal_to_noise",
            processes=1,
            seed=7,
            no_plot=True,
            verbose=False
        )
    )

In [134]:
passed_gensets = list()
for gsea_result in raw_gsea_result:
    passed_gensets.append(
        sorted([(key, value['pval'], value['fdr']) for key, value in gsea_result.results.items() if value['pval'] < 0.05],
               key=lambda x: x[2])
    )

In [135]:
passed_gensets

[[('hsa05217_Basal_cell_carcinoma', 0.0, 0.0),
  ('hsa05224_Breast_cancer', 0.0, 0.0),
  ('hsa05210_Colorectal_cancer', 0.0, 0.09473684210526316),
  ('hsa05225_Hepatocellular_carcinoma', 0.0, 0.18947368421052632)],
 [('hsa05213_Endometrial_cancer', 0.0, 0.0),
  ('hsa05219_Bladder_cancer', 0.0, 0.0),
  ('hsa05222_Small_cell_lung_cancer', 0.0, 0.0),
  ('hsa05225_Hepatocellular_carcinoma', 0.0, 0.0),
  ('hsa05210_Colorectal_cancer', 0.0, 0.019417475728155338),
  ('hsa05215_Prostate_cancer', 0.0, 0.03260869565217391),
  ('hsa05216_Thyroid_cancer', 0.0, 0.09708737864077671)],
 [('hsa05219_Bladder_cancer', 0.0, 0.0)],
 [('hsa05216_Thyroid_cancer', 0.0, 0.0),
  ('hsa05215_Prostate_cancer', 0.0, 0.189873417721519),
  ('hsa05219_Bladder_cancer', 0.0, 0.19891500904159132)],
 [('hsa05211_Renal_cell_carcinoma', 0.0, 0.5591397849462365),
  ('hsa05212_Pancreatic_cancer', 0.0, 0.5591397849462366)],
 [('hsa05222_Small_cell_lung_cancer', 0.0, 0.19402985074626863),
  ('hsa05226_Gastric_cancer', 0.0, 0.3

In [111]:
from scipy import stats
from statsmodels.stats import multitest

In [121]:
def diff_pval(dataset):
    result = dict()
    normal_samples = dataset.index[dataset['sample_type'] == 'norm'].tolist()
    tumor_samples = dataset.index[dataset['sample_type'] == 'cancer'].tolist()
    for gene, row in dataset.iloc[:, :-1].T.iterrows():
        result[gene] = stats.mannwhitneyu(row[normal_samples], row[tumor_samples], alternative='greater').pvalue
    result = pd.Series(result)
    return pd.Series(multitest.multipletests(result)[1], index=result.index)

In [122]:
diff_result = list()
for dataset in tqdm(dataset_list):
    diff_result.append(diff_pval(dataset))

In [131]:
genes_from_gene_sets = set(diff_result[0].index).intersection(
    set().union(*(value for value in target_gensets.values()))
)

In [137]:
ora_result = list()
for diff_data in tqdm(diff_result):
    significant_genes = set(diff_data.index[diff_data < 0.05]).intersection(genes_from_gene_sets)
    geneset_p_values = dict()
    for geneset, genes in target_gensets.items():
        intersection_size = len(set(genes).intersection(significant_genes))
        geneset_p_values[geneset] = stats.fisher_exact(
            [[intersection_size, len(genes) - intersection_size],
             [len(significant_genes), len(genes_from_gene_sets)]],
            alternative='less'
        )[1]
    ora_result.append(geneset_p_values)

In [138]:
ora_result

[{'hsa05210_Colorectal_cancer': 1.0,
  'hsa05211_Renal_cell_carcinoma': 1.0,
  'hsa05212_Pancreatic_cancer': 1.0,
  'hsa05213_Endometrial_cancer': 1.0,
  'hsa05215_Prostate_cancer': 1.0,
  'hsa05216_Thyroid_cancer': 1.0,
  'hsa05217_Basal_cell_carcinoma': 1.0,
  'hsa05219_Bladder_cancer': 1.0,
  'hsa05222_Small_cell_lung_cancer': 1.0,
  'hsa05223_Non-small_cell_lung_cancer': 1.0,
  'hsa05224_Breast_cancer': 1.0,
  'hsa05225_Hepatocellular_carcinoma': 1.0,
  'hsa05226_Gastric_cancer': 1.0},
 {'hsa05210_Colorectal_cancer': 0.8107654766145244,
  'hsa05211_Renal_cell_carcinoma': 0.8789599485811316,
  'hsa05212_Pancreatic_cancer': 0.7051019992651574,
  'hsa05213_Endometrial_cancer': 0.709813470753903,
  'hsa05215_Prostate_cancer': 0.9795550740574336,
  'hsa05216_Thyroid_cancer': 0.650328136522212,
  'hsa05217_Basal_cell_carcinoma': 0.6122890366861372,
  'hsa05219_Bladder_cancer': 0.816569967063365,
  'hsa05222_Small_cell_lung_cancer': 0.642202612655192,
  'hsa05223_Non-small_cell_lung_cance

In [159]:
result = [
    'hsa05224_Breast_cancer',
    'hsa05213_Endometrial_cancer',
    'hsa05219_Bladder_cancer',
    'hsa05216_Thyroid_cancer',
    'hsa05211_Renal_cell_carcinoma',
    'hsa05222_Small_cell_lung_cancer',
    'hsa05211_Renal_cell_carcinoma',
    'hsa05210_Colorectal_cancer',
    'hsa05224_Breast_cancer',
    'hsa05222_Small_cell_lung_cancer',
    'hsa05212_Pancreatic_cancer',
    'hsa05212_Pancreatic_cancer',
    'hsa05216_Thyroid_cancer'
]

In [161]:
print('\n'.join(result))

hsa05224_Breast_cancer
hsa05213_Endometrial_cancer
hsa05219_Bladder_cancer
hsa05216_Thyroid_cancer
hsa05211_Renal_cell_carcinoma
hsa05222_Small_cell_lung_cancer
hsa05211_Renal_cell_carcinoma
hsa05210_Colorectal_cancer
hsa05224_Breast_cancer
hsa05222_Small_cell_lung_cancer
hsa05212_Pancreatic_cancer
hsa05212_Pancreatic_cancer
hsa05216_Thyroid_cancer
